# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key
# Import API key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_pd = pd.read_csv("../output_data/cities.csv")
cities_pd.rename(columns={'years':'Lat'})
cities_pd.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Leningradskiy,100,RU,1586969215,95,69.38,178.42,14.00,4.38
1,Vanimo,32,PG,1586969537,79,-2.67,141.30,80.19,3.44
2,Erzin,75,TR,1586969537,63,36.96,36.20,66.20,9.17
3,Radomir,40,BG,1586969443,56,42.54,22.96,48.99,8.05
4,Mar del Plata,40,AR,1586969194,55,-38.00,-57.56,64.00,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [5]:
# Store latitude and longitude in locations
locations = cities_pd[["Lat", "Lng"]].astype(float)

# Store Humidity values in Humidity
humidity = cities_pd["Humidity"].astype(float)

#Store highest Humidity
max_humidity =cities_pd["Humidity"].max()
print(max_humidity)

100


In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer with maximum humidity for intensity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Ideal weather conditions for me is when, temp is between 85F and 70F, not too humid, cloudy and windy
ideal_weather_df=cities_pd
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"]<85)  & (ideal_weather_df["Max Temp"]>70)]
ideal_weather_df =ideal_weather_df.loc[(ideal_weather_df)["Humidity"]<40 ]
ideal_weather_df =ideal_weather_df.loc[(ideal_weather_df)["Wind Speed"]<20 ]
ideal_weather_df =ideal_weather_df.loc[(ideal_weather_df)["Cloudiness"]<25 ]
ideal_weather_df=ideal_weather_df.dropna()
ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,Mount Isa,0,AU,1586969237,33,-20.73,139.50,77.00,8.05
101,Guerrero Negro,0,MX,1586969228,26,27.98,-114.06,77.16,4.41
108,Redlands,5,US,1586969428,16,34.06,-117.18,73.99,2.84
110,Aswān,0,EG,1586969551,18,24.09,32.91,78.49,9.53
222,Umm Kaddadah,0,SD,1586969268,11,13.60,26.69,77.95,12.15
252,Ladário,0,BR,1586969568,38,-19.00,-57.60,77.00,5.82
470,Viedma,7,AR,1586969072,22,-40.81,-63.00,74.35,14.18
532,Coahuayana Viejo,5,MX,1586969203,26,18.73,-103.68,80.60,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=ideal_weather_df
hotel_df["Hotel Name"]= ""
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(index)
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


23
101
108
110
222
Missing field/result... skipping.
252
470
532


In [9]:
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
23,Mount Isa,0,AU,1586969237,33,-20.73,139.50,77.00,8.05,ibis Styles Mt Isa Verona
101,Guerrero Negro,0,MX,1586969228,26,27.98,-114.06,77.16,4.41,"Casa Laguna, Bed & Breakfast"
108,Redlands,5,US,1586969428,16,34.06,-117.18,73.99,2.84,Dynasty Suites Redlands
110,Aswān,0,EG,1586969551,18,24.09,32.91,78.49,9.53,Nuba Nile
222,Umm Kaddadah,0,SD,1586969268,11,13.60,26.69,77.95,12.15,
252,Ladário,0,BR,1586969568,38,-19.00,-57.60,77.00,5.82,Hotel Fazenda 4 cantos
470,Viedma,7,AR,1586969072,22,-40.81,-63.00,74.35,14.18,Hotel Cristal
532,Coahuayana Viejo,5,MX,1586969203,26,18.73,-103.68,80.60,5.75,Hotel Los Arcos


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content =hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))